# python pdf text parsers test

existed benchmark for english/german language: https://github.com/py-pdf/benchmarks

### список известных:
- PDFMiner -- tested
- PyPDF2 -- tested
- pdfrw -- tested
- slate -- broken(linux)
- tika -- tested
- PDFQuery -- too old
- poppler -- linux
- fitz -- tested
- pdfplumber -- tested

In [2]:
import torch
import os
from pathlib import Path
import warnings
import re

warnings.filterwarnings('ignore')

In [3]:
class PDFDataset(torch.utils.data.Dataset):
    
    def check_pdfs(self):
        self.pdfs = []

        assert Path(f"{self.data_dir}/spbu/pdf").is_dir() == True

        for path in os.listdir(f"{self.data_dir}/spbu/pdf"):
            if os.path.isfile(os.path.join(f"{self.data_dir}/spbu/pdf", path)):
                self.pdfs.append(path)
    
    def __init__(self, data_dir='data'):
        self.data_dir = data_dir
        self.check_pdfs()
    
    def __getitem__(self, idx):
        return {'name': self.pdfs[idx], "path": f"{self.data_dir}/spbu/pdf/{self.pdfs[idx]}"} 
    
    def __len__(self):
        return len(self.pdfs)
    
    def read_pdf(self, idx):
        return open(os.path.join(f"{self.data_dir}/spbu/pdf", self.pdfs[idx]), 'rb')
    

In [4]:
dataset = PDFDataset("data")

# PDFMiner


In [42]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [43]:
output_string = StringIO()

with dataset.read_pdf(1) as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())

Санкт-Петербургский государственный университет

Математическое обеспечение и администрирование информационных
систем
Системное программирование

Леденева Екатерина Юрьевна

Сравнительный анализ алгоритмов
вычисления текстовых метрик для
документации программного обеспечения

Бакалаврская работа

Научный руководитель:
д. т. н., профессор Кознов Д. В.

Рецензент:
руководитель команды аналитики ООО ”Интеллиджей Лабс” Поваров Н. И.

Санкт-Петербург
2020

SAINT-PETERSBURG STATE UNIVERSITY

Software and Administration of Information Systems
Software Engineering

Ekaterina Ledeneva

Comparative analysis of text distance
algorithms for software documentation

Bachelor’s Thesis

Scientific supervisor:
Doctor of Engineering, Professor Dmitry Koznov

Reviewer:
Head of Analytics, IntelliJ Labs Co. Ltd. Nikita Povarov

Saint-Petersburg
2020

Оглавление

Введение

Постановка задачи

1. Обзор

1.1. Форматы документации . . . . . . . . . . . . . . . . . . .
1.2. Duplicate Finder . . . . . . . . . .

## резы

...


# PullEnti

Нет обертки поддерживающей unitext

# PyPDF2

In [44]:
dataset[0]

{'name': 'work_1.pdf', 'path': 'data/spbu/pdf/work_1.pdf'}

In [45]:
from PyPDF2 import PdfReader

reader = PdfReader(dataset[0]['path'])
pages = reader.pages

for page in pages:
    print(page.extract_text())

Санкт­Петербургский государственный университет
Математическое обеспечение и администрирование информационных систем
Системное программирование
Аникин Егор Георгиевич
Реализация расширенного препроцессора для
проекта РуСи
Бакалаврская работа
Научный руководитель:
зав. каф. СП, д.ф.­м.н., проф. А.Н. Терехов
Рецензент:
Директор ООО ”Новые Мобильные Технологии” В.В. Оносовский
Санкт­Петербург
2020
SAINT­PETERSBURG STATE UNIVERSITY
Software and Administration of Information Systems
System Programming
Anikin Egor Georgievich
Implementing an Advanced Preprocessor for a
RuC Project
Bachelor’s Thesis
Scientific supervisor:
chair head, D.Sc, prof. Andrey Terekhov
Reviewer:
CEO LLC New Mobile Technologies Valentin Onossovski
Saint­Petersburg
2020
Содержание
Введение 4
Терминология 7
1. Обзор 8
1.1. Существующие препроцессоры . . . . . . . . . . . . . . . . . 8
1.2. Проект РуСи . . . . . . . . . . . . . . . . . . . . . . . . . . . . 12
2. Архитектура 13
2.1. Функциональные требования к препроцесс

•Вторая — основной блок, каждый компонент которого отвечает за ре­
ализацию близких по смыслу директив:
–calculator отвечает за препроцессорный счёт, выполняющийся по­
средством директивы #eval, которая может быть использована в
других директивах;
–define осуществляет работу с макросами:
*#define — объявление простого макроса,
*#macro — объявление макроса со сложной структурой,
*#undef — удаление простого макроса без параметров,
*#set — изменение простого макроса без параметров;
–if ответственен за работу с ветками условной компиляции с по­
мощью директив #if, #ifdef, #ifundef и #else, а также отвечает за
контроль вложенности блоков условной компиляции;
–while сохраняет препроцессорный цикл #while и воспроизводит
тело цикла, пока выполняется условие;
–include отвечает за добавление содержимого файлов в то место,
где прописана директива #include.
•Третья состоит из одного главного компонента dispatcher, который ини­
циализирует и контролирует всю работу препроцессора. Он обрабаты­
вает 

•put_comma_after_function_ident — после идентификатора в функции долж­
ны быть скобки или запятая и пробел;
•incorrect_use_of_arguments — в функции аргументы должны быть опи­
саны через запятую, в скобках;
•before_endif — перед ’#ENDIF’ должен стоять условный оператор пре­
процессора;
•repeated_ident — этот идентификатор препроцессора уже используется;
•comment_not_closed — длинный комментарий, не закрыт ”/*”;
•excess_parameters — у этой функции больше параметров;
•file_does_not_end_with_define — файл не может закончится до оконча­
ния команды ’#DEFINE’, поставьте перенос строки;
•brackets_not_closed — количество открывающих скобок не соотвеcтву­
ет числу закрывающих;
•put_bracket_after_eval — сразу после команды ’#EV AL’ должен быть
символ открывающейся скобки;
•number_exceeds_limitations — слишком большое число;
•put_digit_after_exp — после экспоненты должно быть число;
•arithmetic_operations_not_allowed — внутри директивы ’#EV AL()’ долж­
ны быть только арифметические выражения;
•lo

# pdfrw

нет извлечения текста

# slate

In [ ]:
!pip install slate

In [32]:
import slate

        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_b39c5b8df3414c1182bcb193c7708e7d\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_b39c5b8df3414c1182bcb193c7708e7d\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppData\Local\Temp\pip-pip-egg-info-upe53qx5'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_b39c5b8df3414c1182bc


The conflict is caused by:
    slate 0.3 depends on distribute
    slate 0.2.3 depends on distribute

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict




      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_096ffc69100d4a91a4ad935b34712587\setuptools\dist.py", line 103
        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_6b25b2a4ef1b4e1db1425175e194c9f0\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_6b25b2a4ef1b4e1db1425175e194c9f0\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppD

ModuleNotFoundError: No module named 'slate'

                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_434f8cc562b442feb56cdace1e53c4bb\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_434f8cc562b442feb56cdace1e53c4bb\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppData\Local\Temp\pip-pip-egg-info-htjy7mkg'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_434f8cc562b442feb56cdace1e53c4bb\
    Complete out

In [ ]:
with dataset.read_pdf(1) as f:
    doc = slate.PDF(f)
    for page in doc:
        print(page)

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_6e066edd33224e26b26e3a6e1c4cae88\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_6e066edd33224e26b26e3a6e1c4cae88\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppData\Local\Temp\pip-pip-egg-info-gbhtayzr'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6e066edd33224e26b26e3a6e1c4cae88\
    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      

    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_5e7abbc8f0db4861907e5d5465021b00\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_5e7abbc8f0db4861907e5d5465021b00\setuptools\extension.py", line 5, in <module>
        from setuptools.dist import _get_unpatched
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_5e7abbc8f0db4861907e5d5465021b00\setuptools\dist.py", line 103
        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\A

    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_ba159e6b95f64dd8b81a1f12630846e9\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_ba159e6b95f64dd8b81a1f12630846e9\setuptools\extension.py", line 2, in <module>
        from setuptools.dist import _get_unpatched
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_ba159e6b95f64dd8b81a1f12630846e9\setuptools\dist.py", line 103
        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\A

        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_a3fa01f60d3e496a84265466243114b9\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_a3fa01f60d3e496a84265466243114b9\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppData\Local\Temp\pip-pip-egg-info-99c3od6_'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_a3fa01f60d3e496a8426

         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6b86b5ca5d6942188614fdb8edc717a2\
    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6b86b5ca5d6942188614fdb8edc717a2\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6b86b5ca5d6942188614fdb8edc717a2\setuptools\extension.py", line 2, in <module>
        from setuptools.dist import _get_unpatched
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6b86b5ca5d6942188614fdb8edc717a2\setuptools\dist.py", line 103
        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\User

      File "<string>", line 1, in <module>
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6bba9bc2056b4eaaadf709a6971b71dc\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6bba9bc2056b4eaaadf709a6971b71dc\setuptools\extension.py", line 2, in <module>
        from setuptools.dist import _get_unpatched
      File "C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_6bba9bc2056b4eaaadf709a6971b71dc\setuptools\dist.py", line 102
        except ValueError, e:
                         ^
    SyntaxError: invalid syntax
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_0034f1c961c44a9bb

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_c4d3d1b7f1ca49cdba3973c5fe1f7c52\\setup.py'"'"'; __file__='"'"'C:\\Users\\ilove\\AppData\\Local\\Temp\\pip-install-lwwgss03\\distribute_c4d3d1b7f1ca49cdba3973c5fe1f7c52\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ilove\AppData\Local\Temp\pip-pip-egg-info-4ntqq0te'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lwwgss03\distribute_c4d3d1b7f1ca49cdba3973c5fe1f7c52\
    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      

нет поддержки с 2017, всё ломается  

похоже нужен линукс

# tika

In [33]:
!pip install tika

  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32642 sha256=d9503fbccf1121168459dd12e1c66f3a798b2f4dfef9f1717b57aea3415ba092
  Stored in directory: c:\users\ilove\appdata\local\pip\cache\wheels\13\56\18\e752060632d32c39c9c4545e756dad281f8504dafcfac02b95
Successfully built tika


In [35]:
from tika import parser as p 
import tika

In [46]:
results = p.from_file(dataset[1]['path'])

In [47]:
results['content']

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nСанкт-Петербургский государственный университет\n\nМатематическое обеспечение и администрирование информационных\nсистем\n\nСистемное программирование\n\nЛеденева Екатерина Юрьевна\n\nСравнительный анализ алгоритмов\nвычисления текстовых метрик для\n\nдокументации программного обеспечения\nБакалаврская работа\n\nНаучный руководитель:\nд. т. н., профессор Кознов Д.В.\n\nРецензент:\nруководитель команды аналитики ООО ”Интеллиджей Лабс” Поваров Н.И.\n\nСанкт-Петербург\n2020\n\n\n\nSAINT-PETERSBURG STATE UNIVERSITY\n\nSoftware and Administration of Information Systems\nSoftware Engineering\n\nEkaterina Ledeneva\n\nComparative analysis of text distance\nalgorithms for software documentation\n\nBachelor’s Thesis\n\nScientific supervisor:\nDoctor of Engineering, Professor Dmitry Koznov\n\nReviewer:\nHead of Analytics, IntelliJ Labs Co. Ltd. Nikita Povarov\n\nSaint-Petersburg\n2020\n\n\n\nОглавление\nВведение 4\n\nПостановка зада

грустно вытаскивать регулярками это всё

# PDFQuery (poppler)

In [49]:
!pip install pdfquery

  Created wheel for pdfquery: filename=pdfquery-0.4.3-py3-none-any.whl size=16799 sha256=734f000b889ff9b08a9a365d26b753a215da318d95e07634c519287db731e1b7
  Stored in directory: c:\users\ilove\appdata\local\pip\cache\wheels\9a\12\58\c1c92713b406092cae17f51b6d67ee96c60675c78e7278eb38
Successfully built pdfquery
  Attempting uninstall: cssselect
    Found existing installation: cssselect 1.1.0
    Uninstalling cssselect-1.1.0:
      Successfully uninstalled cssselect-1.1.0


считывает построчно, я так не хочу

# poppler

In [52]:
import subprocess

In [60]:
def pdftotext(pdf, page=None):
    """Retrieve all text from a PDF file.

    Arguments:
        pdf Path of the file to read.
        page: Number of the page to read. If None, read all the pages.

    Returns:
        A list of lines of text.
    """
    if page is None:
        args = ['pdftotext', '-layout', '-q', pdf, '-']
    else:
        args = ['pdftotext', '-f', str(page), '-l', str(page), '-layout',
                '-q', pdf, '-']
    try:
        print(args)
        txt = subprocess.check_output(args, universal_newlines=True)
        lines = txt.splitlines()
    except subprocess.CalledProcessError:
        lines = []
    return lines

In [56]:
dataset[1]['path']

'data/spbu/pdf/work_10.pdf'

In [69]:
!pip install --use-pep517 python-poppler

  Using cached python_poppler-0.4.1.tar.gz (138 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' 'C:\Users\ilove\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' prepare_metadata_for_build_wheel 'C:\Users\ilove\AppData\Local\Temp\tmp7yvlcg0g'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_12ea7c76d4ac4cb2a096893f4e0a04c3
    Complete output (20 lines):
    + meson setup C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_12ea7c76d4ac4cb2a096893f4e0a04c3 C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_12ea7c76d4ac4cb2a096893f4e0a04c3\.mesonpy-0_jl6ui5\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_12ea7c76d4ac4cb2a096893f4e0a04c3\.mesonpy-0_jl6ui5\build\meson-python-native-file.ini
    The Meson build system
    Version: 1.1.1
    Source dir: C:\Users\ilove\AppData\Local\Temp\pip-inst

  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'error'
  Using cached python_poppler-0.4.0.tar.gz (138 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'error'
  Using cached python-poppler-0.3.0.tar.gz (823 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with statu


    
    A full log can be found at C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_12ea7c76d4ac4cb2a096893f4e0a04c3\.mesonpy-0_jl6ui5\build\meson-logs\meson-log.txt
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\ilove\anaconda3\python.exe' 'C:\Users\ilove\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' prepare_metadata_for_build_wheel 'C:\Users\ilove\AppData\Local\Temp\tmp8qh82szo'
         cwd: C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_9707ca9ed2694a32ae94880aad4673fc
    Complete output (20 lines):
    + meson setup C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_9707ca9ed2694a32ae94880aad4673fc C:\Users\ilove\AppData\Local\Temp\pip-install-lx45lxmi\python-poppler_9707ca9ed2694a32ae94880aad4673fc\.mesonpy-75fq8gfo\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\ilove\AppData\Loc

In [68]:
from poppler import load_from_file, PageRenderer

pdf_document = load_from_file(dataset[1]['path'])
page_1 = pdf_document.create_page(0)
page_1_text = page_1.text()

renderer = PageRenderer()
image = renderer.render_page(page_1)
image_data = image.data

ModuleNotFoundError: No module named 'poppler'

нужен линукс

# fitz

In [71]:
!pip install pymupdf

In [5]:
import fitz

In [6]:
doc = fitz.open(dataset[1]['path'])
page = doc[4]

blocks = page.get_text("dict", flags=16)['blocks']

AttributeError: module 'fitz' has no attribute 'open'

In [200]:
blocks

[{'number': 0,
  'type': 0,
  'bbox': (85.03900146484375,
   47.82830810546875,
   186.0992889404297,
   79.15265655517578),
  'lines': [{'spans': [{'size': 20.662500381469727,
      'flags': 20,
      'font': 'CMUSerif-Bold',
      'color': 0,
      'ascender': 1.1150000095367432,
      'descender': -0.4009999930858612,
      'text': 'Введение',
      'origin': (85.03900146484375, 70.86699676513672),
      'bbox': (85.03900146484375,
       47.82830810546875,
       186.0992889404297,
       79.15265655517578)}],
    'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (85.03900146484375,
     47.82830810546875,
     186.0992889404297,
     79.15265655517578)}]},
 {'number': 1,
  'type': 0,
  'bbox': (85.03900146484375,
   90.53125762939453,
   552.7681274414062,
   725.7260131835938),
  'lines': [{'spans': [{'size': 14.346199989318848,
      'flags': 4,
      'font': 'CMUSerif-Roman',
      'color': 0,
      'ascender': 1.093999981880188,
      'descender': -0.3869999945163727,
      'text

## 

не склеивает текст

# pdfplumber

In [135]:
!pip install pdfplumber

In [141]:
import pdfplumber

with pdfplumber.open(dataset[1]['path']) as pdf:
    for page in pdf.pages:
        print(page.extract_text())

Санкт-Петербургский государственный университет
Математическое обеспечение и администрирование информационных
систем
Системное программирование
Леденева Екатерина Юрьевна
Сравнительный анализ алгоритмов
вычисления текстовых метрик для
документации программного обеспечения
Бакалаврская работа
Научный руководитель:
д.т.н., профессор Кознов Д.В.
Рецензент:
руководитель команды аналитики ООО ”Интеллиджей Лабс” Поваров Н.И.
Санкт-Петербург
2020
SAINT-PETERSBURG STATE UNIVERSITY
Software and Administration of Information Systems
Software Engineering
Ekaterina Ledeneva
Comparative analysis of text distance
algorithms for software documentation
Bachelor’s Thesis
Scientific supervisor:
Doctor of Engineering, Professor Dmitry Koznov
Reviewer:
Head of Analytics, IntelliJ Labs Co. Ltd. Nikita Povarov
Saint-Petersburg
2020
Оглавление
Введение 4
Постановка задачи 6
1. Обзор 7
1.1. Форматы документации . . . . . . . . . . . . . . . . . . . 7
1.2. Duplicate Finder . . . . . . . . . . . . . . . . . . .

ление word2vec [7] или сходство на основе n-грамм из корпуса Google
Web 1T [11].
В данной работе было решено ограничиться синтаксическими ал-
горитмами в виду специфики предметной области: неточные повторы
получаются многократным копированием и изменением текста, а эти
действия являются синтаксическими.
Для исследования были выбраны следующие алгоритмы: наиболь-
шая общая подпоследовательность, косинусное расстояние, расстояние
на основе хеширования и расстояние Левенштейна. Первые три алго-
ритма были взяты из статьи [28], расстояние Левенштейна добавлено в
силу его распространённости.
1.4.1. Наибольшая общая подпоследовательность
Наибольшая общая подпоследовательность (LCS, longest common
subsequence) — набор подстрок, входящих в обе строки в одинаковом
порядке [17]. Как и в [28], мы разрешаем подпоследовательности состо-
ять только их целых слов. Для получения меры сходства из наибольшей
общей подпоследовательности достаточно взять отношение удвоенного
количества слов в ней к сумме 

2.2.2. Сегментация
Сегментация позволяет учитывать структуру JavaDoc-комментариев,
описанную в разделе 1.1. Комментарии разбиваются на JavaDoc-теги,
части описания присваивается пустой тег. Теги, встретившиеся несколь-
ко раз (например, @param, @exception или @throws), группируются: та-
кие теги будем называть составными.
Затем к одноимённым тегам двух комментариев применяется один
из исследуемых алгоритмов.
В случае составных тегов добавляются следующие действия. Обо-
значим за I и J множества вхождений тега в первый и второй коммен-
тарии соответственно. Построим матрицу S = (s ), где s — результат
ij ij
алгоритма на строках i ∈ I и j ∈ J. Чтобы оптимально сопоставить
друг другу элементы I и J и получить итоговое значение, определим
следующую задачу линейного программирования:
max ST · X
∑ X
x ≤1 (∀i ∈ I)
ij
∑j∈J
x ≤1 (∀j ∈ J)
ij
i∈I
где S и X — векторы, полученные из записанных подряд строк соот-
ветствующих матриц. Условия этой задачи означают то, что элементы
I и J сопоставляются 

3.4. Сравнение с результатами статьи Soto и др.
Результаты оценки качества классификации с помощью алгоритмов
LCS, COS и LSH совпали с результатами статьи [28]: и у нас, и в статье
лучшим алгоритмом оказался LCS, а худшим — LSH.
Однако при этом в нашем случае алгоритм LSH не оказался самым
быстрым, как это было в [28]. Это связано с тем, что DITA-топики на-
много длиннее JavaDoc-комментариев, поэтому снижение размерности
данных с помощью LSH даёт большой прирост производительности в
дальнейших вычислениях, тогда как в нашем случае существенными
оказываются затраты на само применение LSH.
3.5. Дискуссия и выводы
Из ответов на четыре вопроса экспериментального исследования
можно сделать несколько выводов.
Лучшим алгоритмом для задачи поиска неточных повторов в
JavaDoc-документации является LCS. При этом другие рассмотренные
алгоритмы ненамного уступают ему в качестве.
Алгоритмы сравнения текста лучше всего использовать в их исход-
ном виде, так как дополнительные шаги приносят мало польз